Here we are going to extract the basketball seasons from the [NBA Site](https://www.nba.com/stats/teams/traditional?Outcome=&SeasonType=Regular+Season&Season=2023-24).

1. To extract dataset from the site, we are going to use a web scraper, and chosen to go with Selenium. <br/>
   Read the docs to know more about [Selenium](https://selenium-python.readthedocs.io/installation.html)


In [257]:
# Importing all neccesary packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from openpyxl import Workbook, load_workbook
from buckets import dimes

In [248]:
# Webdriver: Chrome | Site: NBA site
driver = webdriver.Chrome()
url = "https://www.nba.com/stats/teams/traditional/"
driver.get(url)

In [249]:
# Create an Excel to begin data transfer
wb = Workbook()
wb.save('nba_stats.xlsx')
ws = wb.active
dfs = []

In [250]:
# Capture all seasons in the list from web page
season_drpdwn = Select(driver.find_element(By.CLASS_NAME,"DropDown_select__4pIg9"))
seasons_arr = [sn.text for sn in season_drpdwn.options]
xpath_table = dimes.get('XPATH')

In [251]:
# Fetch and Export Seasons' dataset into Excel
for i,sn in enumerate(seasons_arr):
    season_drpdwn.select_by_visible_text(sn)
    table = WebDriverWait(driver, 40).until(EC.visibility_of_element_located((By.XPATH, xpath_table))).get_attribute("outerHTML")
    df = pd.read_html(table)[0]
    df.dropna(how='all', axis=1, inplace=True)
    df.columns.values[0] = 'Rank'
    df.insert(2, "Season", sn)
    # print(df)
    dfs.append(df)
    with pd.ExcelWriter(path='nba_stats.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
       df.to_excel(writer, sheet_name= sn, index=False, header=True)



In [256]:
# Combine all df and sorted by Win Rate
full_df = pd.concat([x for x in dfs], ignore_index=True)
full_df = full_df.sort_values(by="WIN%", ascending= False).reset_index(drop=True)
full_df["Rank"] = full_df.index + 1
with pd.ExcelWriter(path='nba_stats.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    full_df.to_excel(writer, sheet_name= 'Sheet', index=False, header=True)

In [253]:
# Close all opened files and systems
wb.close()
driver.close()

In [260]:
full_df.describe()

,Rank,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
count,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,...,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000
mean,416.500000,79.153846,39.576923,39.576923,0.499837,48.347356,101.415745,37.764784,83.027885,45.467668,...,31.443630,42.635817,22.435216,14.517067,7.664663,4.913221,4.913942,20.987139,14.118029,-0.005048
std,240.322006,7.041249,12.644595,12.584209,0.152629,0.178016,7.750690,2.673842,4.263605,1.721710,...,2.343331,2.091980,2.371430,1.217105,0.887214,0.816684,0.706424,1.802990,9.427177,4.621300
min,1.000000,50.000000,7.000000,9.000000,0.106000,48.000000,81.900000,30.800000,71.200000,40.100000,...,24.900000,35.800000,15.600000,11.100000,5.500000,2.400000,3.000000,15.800000,0.000000,-13.900000
25%,208.750000,82.000000,30.000000,30.000000,0.390000,48.200000,95.675000,35.900000,79.800000,44.300000,...,29.700000,41.200000,20.800000,13.700000,7.100000,4.300000,4.400000,19.700000,0.200000,-3.100000
50%,416.500000,82.000000,41.000000,39.000000,0.512000,48.400000,99.900000,37.400000,82.600000,45.400000,...,31.250000,42.600000,22.100000,14.500000,7.600000,4.900000,4.900000,20.900000,19.500000,0.300000
75%,624.250000,82.000000,49.000000,49.000000,0.610000,48.500000,106.600000,39.500000,86.200000,46.600000,...,33.100000,44.000000,23.900000,15.225000,8.200000,5.400000,5.400000,22.200000,20.800000,3.300000
max,832.000000,82.000000,73.000000,72.000000,0.890000,49.000000,123.300000,46.900000,94.400000,50.700000,...,42.200000,51.700000,30.700000,19.000000,12.000000,8.200000,6.900000,27.100000,25.700000,11.600000


2. Now that we have extracted our data into a df we're going to 
